In [1]:
import csv, re, time, math
from datetime import date, datetime, timedelta, time as t
import pandas as pd
import numpy as np
from scipy.stats import norm
from kiteconnect import KiteConnect, exceptions, KiteTicker
from zerodha import Zerodha
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from scipy.optimize import newton
from scipy.optimize import brentq
from math import log, sqrt, exp

import warnings
warnings.filterwarnings("ignore")

In [2]:
expiry = '2024-05-30'   ##change expiry
expiry_1 = datetime(2024,5,30,15,30,0)  ##change expiry

time_1 = t(9,15)
time_2 = t(15,30)
exit_time = t(15,0)

In [3]:
#variables for 09:15 values
expiry1 = datetime(2024, 5, 30)  # change expiry date

current_datetime = datetime.now()
current_datetime = current_datetime.replace(hour=9, minute=15, second=0, microsecond=0)
time_to_maturity_days = (expiry_1 - current_datetime).days
time_to_maturity_years = time_to_maturity_days / 365

implied_volatility = 0.2
risk_free_rate = 0.07
dividend_yield = 0.02

today = datetime.now().strftime('%Y-%m-%d')

sum_ce_vega_change = 0.0
sum_pe_vega_change = 0.0

In [4]:
kite = Zerodha(user_id='NX5644', password='Sanjay400@', twofa='Z2RKD6DPOV3GRL3DKLOFU44I76G3LVMI')

kite.login()

{'status': 'success', 'data': {'profile': {}}}


{'status': 'success', 'data': {'profile': {}}}

In [5]:
for j in range(0, 1):
    #try:
    instruments = pd.DataFrame(kite.instruments())
    instruments.to_csv('instruments.csv')
    
instruments = pd.read_csv('instruments.csv')
instruments1 = instruments[(instruments.exchange == 'NSE') & (instruments.name == 'NIFTY 50') & (instruments.segment == 'INDICES')]
underlying_inst_id = instruments1.iloc[0]['instrument_token']

In [6]:
nifty_data = kite.historical_data(instrument_token = underlying_inst_id, from_date=today, to_date=today, interval="day")
open_pric = nifty_data[0]['open']

In [7]:
#greeks calculation

def d1(up, sp, t, r, v, d):
    return (math.log(up / sp) + (r - d + 0.5 * v ** 2) * t) / (v * math.sqrt(t))

def nd1(up, sp, t, r, v, d):
    return math.exp(-(d1(up, sp, t, r, v, d) ** 2) / 2) / (math.sqrt(2 * math.pi))

def d2(up, sp, t, r, v, d):
    return d1(up, sp, t, r, v, d) - v * math.sqrt(t)

def nd2(up, sp, t, r, v, d):
    return 0.5 * (1 + math.erf(d2(up, sp, t, r, v, d) / math.sqrt(2)))

def call_price(up, sp, t, r, v, d):
    return math.exp(-d * t) * up * nd1(up, sp, t, r, v, d) - sp * math.exp(-r * t) * nd1(up, sp, t, r, v, d - v * math.sqrt(t))

def put_price(up, sp, t, r, v, d):
    return sp * math.exp(-r * t) * (1 - nd2(up, sp, t, r, v, d)) - math.exp(-d * t) * up * (1 - nd1(up, sp, t, r, v, d))

def call_delta(up, sp, t, r, v, d):
    return nd1(up, sp, t, r, v, d)

def put_delta(up, sp, t, r, v, d):
    return nd1(up, sp, t, r, v, d) - 1

def call_theta(up, sp, t, r, v, d):
    ct = -((up * v * nd1(up, sp, t, r, v, d)) / (2 * math.sqrt(t)) - r * sp * math.exp(-r * t) * nd2(up, sp, t, r, v, d))
    return ct / 365

def option_gamma(up, sp, t, r, v, d):
    return nd1(up, sp, t, r, v, d) / (up * v * math.sqrt(t))

def option_vega(up, sp, t, r, v, d):
    return 0.01 * up * math.sqrt(t) * nd1(up, sp, t, r, v, d)

def put_theta(up, sp, t, r, v, d):
    pt = -(up * v * nd1(up, sp, t, r, v, d)) / (2 * math.sqrt(t)) + r * sp * math.exp(-r * t) * (1 - nd2(up, sp, t, r, v, d))
    return pt / 365

def call_rho(up, sp, t, r, v, d):
    return 0.01 * sp * t * math.exp(-r * t) * nd2(up, sp, t, r, v, d)

def put_rho(up, sp, t, r, v, d):
    return -0.01 * sp * t * math.exp(-r * t) * (1 - nd2(up, sp, t, r, v, d))

def call_iv(up, sp, t, r, mp, d):
    mx = 5
    mn = 0
    while (mx - mn) > 0.0001:
        if call_price(up, sp, t, r, (mx + mn) / 2, d) > mp:
            mx = (mx + mn) / 2
        else:
            mn = (mx + mn) / 2
    return (mx + mn) / 2

def put_iv(up, sp, t, r, mp, d):
    mx = 5
    mn = 0
    while (mx - mn) > 0.0001:
        if put_price(up, sp, t, r, (mx + mn) / 2, d) > mp:
            mx = (mx + mn) / 2
        else:
            mn = (mx + mn) / 2
    return (mx + mn) / 2

def black_scholes_call(S, K, T, r, q, sigma):
    d1 = (log(S / K) + (r - q + sigma**2 / 2) * T) / (sigma * sqrt(T))
    d2 = d1 - sigma * sqrt(T)
    call_price = S * exp(-q * T) * norm.cdf(d1) - K * exp(-r * T) * norm.cdf(d2)
    return call_price

def black_scholes_put(S, K, T, r, q, sigma):
    d1 = (log(S / K) + (r - q + sigma**2 / 2) * T) / (sigma * sqrt(T))
    d2 = d1 - sigma * sqrt(T)
    put_price = K * exp(-r * T) * norm.cdf(-d2) - S * exp(-q * T) * norm.cdf(-d1)
    return put_price


def calculate_iv(option_price, S, K, T, r, q, option_type):
    if option_type == 'call':
        option_func = lambda sigma: black_scholes_call(S, K, T, r, q, sigma) - option_price
    elif option_type == 'put':
        option_func = lambda sigma: black_scholes_put(S, K, T, r, q, sigma) - option_price
    else:
        raise ValueError("Invalid option type")

    
    a = 0.01  
    b = 2.0   
    
    
    if option_func(a) * option_func(b) >= 0:
        raise ValueError("Function does not change sign in the interval [a, b]")

    return brentq(option_func, a, b)

def option_vomma(up, sp, t, r, v, d):
    d1_val = d1(up, sp, t, r, v, d)
    nd1_val = nd1(up, sp, t, r, v, d)
    return 0.01 * up * math.sqrt(t) * d1_val * nd1_val


In [9]:
while True:
    now = datetime.now()
    if time_1 < t(datetime.now().time().hour, datetime.now().time().minute) < time_2 and now.second == 0:
        vega_sum = 0
        vega_sum_pe = 0
        vomma_sum = 0
        vomma_sum_pe = 0
       
        underlying_price = kite.historical_data(instrument_token = underlying_inst_id, from_date=today, to_date=now, interval="day")
        underlying_price = pd.DataFrame(underlying_price)
        underlying_price = underlying_price['close'][0]
        
        atm_live = round(underlying_price / 50) * 50
          
        year_to_expiry = (expiry_1 - datetime.now())/timedelta(days=1)/365
        
        strike_prices_ce = [atm_live+50, atm_live+100, atm_live+150, atm_live+200, atm_live+250, atm_live+300, atm_live+350, atm_live+400, atm_live+450]
        strike_prices_pe = [atm_live-50, atm_live-100, atm_live-150, atm_live-200, atm_live-250, atm_live-300, atm_live-350, atm_live-400, atm_live-450]
        
        for strike in strike_prices_ce:
            instrum = instruments[(instruments.exchange == 'NFO') & (instruments.name == 'NIFTY') & (instruments.instrument_type == 'CE') & (instruments.strike == strike)]
            instrum = instrum[instrum.expiry == expiry]
            instrum = int(instrum.iloc[0]['instrument_token'])
        
            ltp = kite.historical_data(instrument_token = instrum, from_date=today, to_date=now, interval="day")
            ltp = pd.DataFrame(ltp)
            premium = ltp['close'][0]
    
            iv_live = calculate_iv(premium, underlying_price, strike, year_to_expiry, risk_free_rate, dividend_yield, 'call')
            vega_live = option_vega(underlying_price, strike, year_to_expiry, risk_free_rate, iv_live, dividend_yield)
            vomma_live = option_vomma(underlying_price, strike, year_to_expiry, risk_free_rate, iv_live, dividend_yield)
        
            #initial value
            ce_open = kite.historical_data(instrument_token = instrum, from_date=today, to_date=today, interval="day")
            ce_open = ce_open[0]['open']
            ce_iv_initial = calculate_iv(ce_open, open_pric, strike, time_to_maturity_years, risk_free_rate, dividend_yield, 'call')
            ce_vega_initial = option_vega(open_pric, strike, time_to_maturity_years, risk_free_rate, ce_iv_initial, dividend_yield)
            ce_vomma_initial = option_vomma(open_pric, strike, time_to_maturity_years, risk_free_rate, ce_iv_initial, dividend_yield)
                
            ce_vega_ch = (vega_live - ce_vega_initial)
            vega_sum += ce_vega_ch
            
            ce_vomma_ch = (vomma_live - ce_vomma_initial)
            vomma_sum += ce_vomma_ch
            
        for strikes in strike_prices_pe:
            instrum_pe = instruments[(instruments.exchange == 'NFO') & (instruments.name == 'NIFTY') & (instruments.instrument_type == 'PE') & (instruments.strike == strikes)]
            instrum_pe = instrum_pe[instrum_pe.expiry == expiry]
            instrum_pe = int(instrum_pe.iloc[0]['instrument_token'])

            ltp = kite.historical_data(instrument_token = instrum_pe, from_date=today, to_date=now, interval="day")
            ltp = pd.DataFrame(ltp)
            premium_pe = ltp['close'][0]
    
            iv_live_pe = calculate_iv(premium_pe, underlying_price, strikes, year_to_expiry, risk_free_rate, dividend_yield, 'put')
            vega_live_pe = option_vega(underlying_price, strikes, year_to_expiry, risk_free_rate, iv_live_pe, dividend_yield)
            vomma_live_pe = option_vomma(underlying_price, strikes, year_to_expiry, risk_free_rate, iv_live_pe, dividend_yield)

            #initial value
            pe_open = kite.historical_data(instrument_token = instrum_pe, from_date=today, to_date=today, interval="day")
            pe_open = pe_open[0]['open']
            pe_iv_initial = calculate_iv(pe_open, open_pric, strikes, time_to_maturity_years, risk_free_rate, dividend_yield, 'put')
            pe_vega_initial = option_vega(open_pric, strikes, time_to_maturity_years, risk_free_rate, pe_iv_initial, dividend_yield)
            pe_vomma_initial = option_vomma(open_pric, strikes, time_to_maturity_years, risk_free_rate, pe_iv_initial, dividend_yield)

            
            pe_vega_ch = (vega_live_pe - pe_vega_initial)
            vega_sum_pe += pe_vega_ch
            
            pe_vomma_ch = (vomma_live_pe - pe_vomma_initial)
            vomma_sum_pe += pe_vomma_ch
            
        print(datetime.now(), vega_sum, vega_sum_pe, vomma_sum, vomma_sum_pe)
    
        csv_file = f"vega_vomma_data{today}.csv"
        try:
            with open(csv_file, 'x', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(['Timestamp', 'CE_Vega', 'PE_Vega', 'CE_Vomma', 'PE_Vomma'])
        except FileExistsError:
            pass

            timestamp = time.strftime('%Y-%m-%d %H:%M:%S')
            with open(csv_file, 'a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([timestamp, vega_sum, vega_sum_pe, vomma_sum, vomma_sum_pe])
    
    
        

2024-05-23 09:50:07.176581 9.553821982475494 -4.596414310647236 5.857961232501124 10.961478314620496
2024-05-23 09:51:06.167937 8.616787558623106 -5.207167364128968 7.732092653099464 8.572187766338214
2024-05-23 09:52:06.157814 10.290759286012314 -5.229383870718669 6.754832326031225 11.858192258253123
2024-05-23 09:53:06.889149 10.036339661684725 -4.999392704777365 6.4311990980878955 11.5177382364725
2024-05-23 09:54:06.141223 9.72056494875001 -4.620536589330135 6.034298290478651 11.028086855722275
2024-05-23 09:55:03.441905 8.434575859395682 -5.249446032179113 7.499020686230068 8.54178517065203
2024-05-23 09:56:03.616595 7.878027172794169 -4.647902993284469 6.703339215482348 7.991299914144227


TokenException: Incorrect `api_key` or `access_token`.